In [1]:
using Pkg
Pkg.activate("D://ATIS3//Project.toml") # change path 
Pkg.instantiate()

LoadError: StackOverflowError:

In [2]:
using JuMP, LinearAlgebra, Plots, StatsPlots, CPLEX, Statistics, CSV, DataFrames

In [3]:
#read scenario data
scenarios = CSV.read("scenarios_multinomial.csv", DataFrame)

SOC = 0.33
hexagons = names(scenarios)[1:length(names(scenarios))-1] # vector of hexagons (strings)
hours = 4 # duration for charging
car_kWh = 30 # capacity of each car

#demand mxn (m=hexagon, n=scenario) in kWh summe über car*max_kWh*(1-SOC)
demand_cars = transpose(convert(Matrix,scenarios[:,hexagons]))
demand_kWh = demand_cars.*car_kWh.*(1-SOC)

probability = transpose(scenarios.probs) # probability for each scenario

P = [3, 7, 11] # different charger types in kW
charger_cost = [1000, 800, 500] # investment cost for charger 3 kW, 7 kW und 11 kW in €/kW
opportunity_cost = 5000 # cost of demand not served


5000

In [4]:
function charger_siting(SOC, hexagons, hours, car_kWh, demand_cars, demand_kWh, probability, P,
        charger_cost, opportunity_cost)
    
    # Define Sets
    Γ = Array{Int}(1:size(hexagons,1)) # Set with hexagons
    Σ = Array{Int}(1:size(P,1)) # Set with charger types 
    Ω = Array{Int}(1:size(probability,1)) # Set with scenarios

    # Initialize model    
    m = Model(CPLEX.Optimizer)

    # Define variables
    @variable(m, 0 <= x_charger[i=Γ, j=Σ], Int) # number of specific charger type in each hexagon
    @variable(m, 0 <= E_supplied[i=Γ, j=Σ, k=Ω]) # supplied energy in each hexagon kWh
    @variable(m, 0 <= E_not_supplied[i=Γ, k=Ω]) # demand not covered kWh
    @variable(m, 0 <= cars_supplied[i=Γ, j=Σ, k=Ω], Int)
    @variable(m, 0 <= cars_not_supplied[i=Γ, j=Σ, k=Ω], Int)

    # Objective function 
    @expression(m, invest_cost[i=Γ,j=Σ], charger_cost[j].*x_charger[i,j].*P[j]) # first stage investment cost
    @expression(m, charging_cost[i=Γ,k=Ω], opportunity_cost.*E_not_supplied[i,k].*probability[k])
    @objective(m, Min, sum(invest_cost) 
                     + sum(charging_cost)
                      )

    # Constraints
    # Eq. Balance equation: Demand in each hexagon has to equal demand. Loss of load is possible
    @constraint(m, eq_balance[i=Γ,k=Ω], sum(E_supplied[i,j,k] for j in Σ) + E_not_supplied[i,k] == demand_kWh[i,k])

    # Eq. energy limit: rated_power*chargingtime must not exeed energy
    @constraint(m, eq_cap[i=Γ,j=Σ,k=Ω], E_supplied[i,j,k] <= x_charger[i,j].*P[j].*hours)

    # Eq. supplied cars and not supplied cars have to equal amount of cars in each hexagon
    @constraint(m, eq_cars_supplied[i=Γ,j=Σ,k=Ω], sum(cars_supplied[i,j,k] for j in Σ) 
                                                + sum(cars_not_supplied[i,j,k] for j in Σ)
                                                == demand_cars[i,k])

    # Eq. maximum one car per charging station
    @constraint(m, eq_max_cars[i=Γ,j=Σ,k=Ω], cars_supplied[i,j,k] <= x_charger[i,j])

    # Eq. 
    @constraint(m, eq_max_supply[i=Γ,j=Σ,k=Ω], E_supplied[i,j,k] <= cars_supplied[i,j,k].*car_kWh.*(1-SOC))


    # Run optimization
    optimize!(m)

    # Evaluate resuluts
    obj = objective_value(m)
    charger_in_hex = value.(x_charger)
    
    return charger_in_hex, x_charger, E_not_supplied, E_supplied, cars_not_supplied, cars_supplied
end


charger_siting (generic function with 1 method)

In [5]:
charger_in_hex, x_charger, E_not_supplied, E_supplied, cars_not_supplied, cars_supplied = charger_siting(
    SOC, hexagons, hours, car_kWh, demand_cars, demand_kWh, probability, P, charger_cost, opportunity_cost)

println(charger_in_hex)

Version identifier: 12.10.0.0 | 2019-11-26 | 843d4de2ae
Found incumbent of value 1.6258500e+07 after 0.00 sec. (0.10 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 480 rows and 480 columns.
MIP Presolve modified 10 coefficients.
Reduced MIP has 781 rows, 781 columns, and 1846 nonzeros.
Reduced MIP has 12 binaries, 485 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.03 sec. (1.47 ticks)
Probing time = 0.00 sec. (0.02 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 770 rows and 770 columns.
Reduced MIP has 11 rows, 11 columns, and 26 nonzeros.
Reduced MIP has 0 binaries, 7 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.40 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 11 rows, 11 columns, and 26 nonzeros.
Reduced MIP has 0 binaries, 7 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.02 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, u

In [6]:
hours_lst = [2, 3, 4, 5, 6, 7, 8]
hours_dict = Dict()
energy_dict = Dict()
for hours in hours_lst
    charger_in_hex, x_charger, E_not_supplied, = charger_siting(
        SOC, hexagons, hours, car_kWh, demand_cars, demand_kWh, probability, P, charger_cost, opportunity_cost)
    hours_dict[hours] = charger_in_hex
    energy_dict[hours] = value.(E_not_supplied)
end

Version identifier: 12.10.0.0 | 2019-11-26 | 843d4de2ae
Found incumbent of value 1.6258500e+07 after 0.00 sec. (0.10 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 480 rows and 480 columns.
MIP Presolve modified 2 coefficients.
Reduced MIP has 781 rows, 781 columns, and 1846 nonzeros.
Reduced MIP has 10 binaries, 487 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (1.47 ticks)
Probing time = 0.00 sec. (0.02 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 770 rows and 770 columns.
Reduced MIP has 11 rows, 11 columns, and 26 nonzeros.
Reduced MIP has 0 binaries, 7 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.40 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 11 rows, 11 columns, and 26 nonzeros.
Reduced MIP has 0 binaries, 7 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.02 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, us

In [7]:
for hours in hours_lst
    println(hours, " hours: ", "charger_in_hex = ", sum(hours_dict[hours]),
        " / Energy not supplied = ", sum(energy_dict[hours]))
end

2 hours: charger_in_hex = 1500.0 / Energy not supplied = 0.0
3 hours: charger_in_hex = 1500.0 / Energy not supplied = 0.0
4 hours: charger_in_hex = 2487.0 / Energy not supplied = 137.0999999999984
5 hours: charger_in_hex = 2007.0 / Energy not supplied = 149.99999999999818
6 hours: charger_in_hex = 1680.0 / Energy not supplied = 121.49999999599868
7 hours: charger_in_hex = 1500.0 / Energy not supplied = 0.0
8 hours: charger_in_hex = 1500.0 / Energy not supplied = 0.0


In [8]:
hours

4

In [11]:
op_cost_lst = [3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000]
op_cost_dict = Dict()
energy_dict = Dict()
for opportunity_cost in op_cost_lst
    charger_in_hex, x_charger, E_not_supplied, = charger_siting(
        SOC, hexagons, hours, car_kWh, demand_cars, demand_kWh, probability, P, charger_cost, opportunity_cost)
    op_cost_dict[opportunity_cost] = charger_in_hex
    energy_dict[opportunity_cost] = value.(E_not_supplied)
end

Version identifier: 12.10.0.0 | 2019-11-26 | 843d4de2ae
Found incumbent of value 1.1555100e+07 after 0.00 sec. (0.10 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 480 rows and 480 columns.
MIP Presolve modified 10 coefficients.
Reduced MIP has 781 rows, 781 columns, and 1846 nonzeros.
Reduced MIP has 12 binaries, 485 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (1.47 ticks)
Probing time = 0.00 sec. (0.02 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 770 rows and 770 columns.
Reduced MIP has 11 rows, 11 columns, and 26 nonzeros.
Reduced MIP has 0 binaries, 7 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.40 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 11 rows, 11 columns, and 26 nonzeros.
Reduced MIP has 0 binaries, 7 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.02 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, u

3000 Opportunity Cost: charger_in_hex = 0.0 / Energy not supplied = 30150.000000000015
4000 Opportunity Cost: charger_in_hex = 2486.0 / Energy not supplied = 188.39999998999792
5000 Opportunity Cost: charger_in_hex = 2487.0 / Energy not supplied = 137.0999999999984
6000 Opportunity Cost: charger_in_hex = 2491.0 / Energy not supplied = 97.19999999999858
7000 Opportunity Cost: charger_in_hex = 2476.0 / Energy not supplied = 34.49999999999874
8000 Opportunity Cost: charger_in_hex = 2477.0 / Energy not supplied = 20.999999999998998
9000 Opportunity Cost: charger_in_hex = 2477.0 / Energy not supplied = 20.999999999999083
10000 Opportunity Cost: charger_in_hex = 2477.0 / Energy not supplied = 20.999999999999083


In [21]:
for opportunity_cost in op_cost_lst
    println(opportunity_cost, " Opportunity Cost: ", "charger_in_hex = ", sum(op_cost_dict[opportunity_cost]),
        " / Energy not supplied = ", sum(energy_dict[opportunity_cost]))
end

3000 Opportunity Cost: charger_in_hex = 0.0 / Energy not supplied = 30150.000000000015
4000 Opportunity Cost: charger_in_hex = 2486.0 / Energy not supplied = 188.39999998999792
5000 Opportunity Cost: charger_in_hex = 2487.0 / Energy not supplied = 137.0999999999984
6000 Opportunity Cost: charger_in_hex = 2491.0 / Energy not supplied = 97.19999999999858
7000 Opportunity Cost: charger_in_hex = 2476.0 / Energy not supplied = 34.49999999999874
8000 Opportunity Cost: charger_in_hex = 2477.0 / Energy not supplied = 20.999999999998998
9000 Opportunity Cost: charger_in_hex = 2477.0 / Energy not supplied = 20.999999999999083
10000 Opportunity Cost: charger_in_hex = 2477.0 / Energy not supplied = 20.999999999999083


In [22]:
opportunity_cost

5000

In [38]:
charger_cost = [600, 400, 100]# investment cost for charger 3 kW, 7 kW und 11 kW in €/kW
charger_cost_delta = 600
charger_cost_dict = Dict()
energy_dict = Dict()
for i in range(0, step=100, stop=charger_cost_delta)
    charger_in_hex, x_charger, E_not_supplied, = charger_siting(
        SOC, hexagons, hours, car_kWh, demand_cars, demand_kWh, probability, P, charger_cost.+i, opportunity_cost)
    charger_cost_dict[charger_cost.+i] = charger_in_hex
    energy_dict[charger_cost.+i] = value.(E_not_supplied)
end

Version identifier: 12.10.0.0 | 2019-11-26 | 843d4de2ae
Found incumbent of value 1.4458500e+07 after 0.00 sec. (0.10 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 480 rows and 480 columns.
MIP Presolve modified 10 coefficients.
Reduced MIP has 781 rows, 781 columns, and 1846 nonzeros.
Reduced MIP has 12 binaries, 485 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (1.47 ticks)
Probing time = 0.00 sec. (0.02 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 770 rows and 770 columns.
Reduced MIP has 11 rows, 11 columns, and 26 nonzeros.
Reduced MIP has 0 binaries, 7 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.40 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 11 rows, 11 columns, and 26 nonzeros.
Reduced MIP has 0 binaries, 7 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.02 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, u

In [49]:
for key in keys(charger_cost_dict)
    println(key, " Charger Cost: ", "charger_in_hex = ", sum(charger_cost_dict[key]),
        " / Energy not supplied = ", sum(energy_dict[key]))
end

[1200, 1000, 700] Charger Cost: charger_in_hex = 2491.0000000001664 / Energy not supplied = 279.29999999799804
[900, 700, 400] Charger Cost: charger_in_hex = 1500.0 / Energy not supplied = 0.0
[600, 400, 100] Charger Cost: charger_in_hex = 1500.0 / Energy not supplied = 0.0
[800, 600, 300] Charger Cost: charger_in_hex = 1500.0 / Energy not supplied = 0.0
[700, 500, 200] Charger Cost: charger_in_hex = 1500.0 / Energy not supplied = 0.0
[1000, 800, 500] Charger Cost: charger_in_hex = 2487.0 / Energy not supplied = 137.0999999999984
[1100, 900, 600] Charger Cost: charger_in_hex = 2493.0000000001664 / Energy not supplied = 172.49999999299817
